### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('original_movies.dat',
                     delimiter='::',
                     header=None,
                     names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')

reviews = pd.read_csv('original_ratings.dat',
                      delimiter='::',
                      header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object},
                      engine='python')

# Reduce the size reviews dataset
reviews = reviews.loc[:100000,:]

In [12]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [11]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [25]:
movies['genre'].nunique()

2736

In [29]:
movies['genre'].isna().any()

True

In [38]:
movies['genre'].str.split('|', expand = True)

,0,1,2,3,4,5,6,7,8,9
0,Documentary,Short,None,None,None,None,None,None,None,None
1,Documentary,Short,None,None,None,None,None,None,None,None
2,Documentary,Short,None,None,None,None,None,None,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Short,Horror,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
35474,Documentary,Adventure,Biography,Family,None,None,None,None,None,None
35475,Documentary,Music,None,None,None,None,None,None,None,None
35476,Documentary,None,None,None,None,None,None,None,None,None
35477,Documentary,None,None,None,None,None,None,None,None,None


In [42]:
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass
    
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))

The number of genres is 28.


#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [27]:
# Use this cell to find the following information:
# number of movies
print('Number of movies:',movies.shape[0])
# number of ratings
print('Number of ratings:', reviews.shape[0] )
# number of different genres
print('Number of genres', movies['genre'].nunique())
# number of unique users
print('Number of unique users:', reviews['user_id'].nunique())
# number of missing ratings
print('Number of missing ratings', reviews['rating'].isna().sum())
# the `average`, `min`, and `max` ratings given
reviews.describe()

Number of movies: 35479
Number of ratings: 100001
Number of genres 2736
Number of unique users: 8022
Number of missing ratings 0


,rating
count,100001.000000
mean,7.397666
std,1.833944
min,0.000000
25%,6.000000
50%,8.000000
75%,9.000000
max,10.000000


In [ ]:
# You can click `+` button on the navbar to add cells for additional work

In [43]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 8022
b = 10
c = 7
d = 35479
e = 15
f = 0
g = 4
h = 100001
i = 28

# Guide: replace "ENTER YOUR ANSWER HERE" with the matching letter above.
# For example, 'The number of movies in the dataset': d,

dict_sol1 = {
'The number of movies in the dataset': d, 
'The number of ratings in the dataset': h,
'The number of different genres': i, 
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f, 
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's for each genre

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [45]:
# Cell for work
movies.tail()

,movie_id,movie,genre
35474,12036052,Elephant (2020),Documentary|Adventure|Biography|Family
35475,12058882,Coachella: 20 Years in the Desert (2020),Documentary|Music
35476,12078990,Money Heist: The Phenomenon (2020),Documentary
35477,12131262,Out of Shadows (2020),Documentary
35478,12192654,Planet of the Humans (2019),NaN


In [53]:
movies['movie'][1][-5:-1]

'1895'

In [60]:
year = movies['movie'].apply(lambda st: st[-5:-1])
movies['year'] = year

In [61]:
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['year'].apply(add_movie_year)
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [63]:
def split_genres(val):
    try:
        if val.find(g) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0
for g in genres:
    movies[g] = movies['genre'].apply(split_genres)
    
    
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's,Action,Music,Fantasy,...,Adventure,Documentary,Reality-TV,Comedy,Horror,Family,Film-Noir,Drama,News,Sport
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [64]:
# Click `+` button on the nav menu to add more cells for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [65]:
import datetime
change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')
reviews['date_time'] = reviews['timestamp'].apply(change_timestamp)
reviews.head()

,user_id,movie_id,rating,timestamp,date_time
0,1,0114508,8,1381006850,2013-10-05 23:00:50
1,2,0208092,5,1586466072,2020-04-09 23:01:12
2,2,0358273,9,1579057827,2020-01-15 04:10:27
3,2,10039344,5,1578603053,2020-01-09 21:50:53
4,2,6751668,9,1578955697,2020-01-13 23:48:17


In [66]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date
0           0        1    114508       8  1381006850  2013-10-05 21:00:50
1           1        2    208092       5  1586466072  2020-04-09 21:01:12
2           2        2    358273       9  1579057827  2020-01-15 03:10:27
3           3        2  10039344       5  1578603053  2020-01-09 20:50:53
4           4        2   6751668       9  1578955697  2020-01-13 22:48:17
   Unnamed: 0  movie_id                                              movie  \
0           0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1           1        10                La sortie des usines Lumière (1895)   
2           2        12                      The Arrival of a Train (1896)   
3           3        25  The Oxford and Cambridge University Boat Race ...   
4           4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  Documentary  Adventure  \
0  Docume